# Detector de imagenes (Fumadores)

## Importación del dataset

In [1]:
!apt-get install unrar
!pip install rarfile

"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import rarfile

rarfile.UNRAR_TOOL = "/usr/bin/unrar"
!wget https://github.com/repositoriosHackaton/SIC25es-Remember-Us-Recuerdanos-/raw/refs/heads/main/recursos/dataset.rar

rar_path = "/content/dataset.rar"  # Ruta del archivo RAR
extract_path = "dataset"  # Carpeta de salida

with rarfile.RarFile(rar_path) as rar_ref:
    rar_ref.extractall(extract_path)

print("Archivo descomprimido correctamente.")

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset.rar'

## Entrenamiento de modelos

### SVC - Sklearn

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from PIL import Image
import numpy as np
import os

#### Tratamiento de datos

In [ ]:
# Función para cargar las imagenes desde el folder y almacenarlas en forma de vector númerico, junto con otro array con su clasificación
def svc_loadImages(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename)).convert("L")
        img = img.resize((250,250))
        img_array = np.array(img).flatten()
        images.append(img_array)
        label = 0 if "notsmoking" in filename else 1 # Si el nombre de la imagen es 'notsmoking' colocar 0, caso contrario 1
        labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
#Separacion imagenes train y test
scv_Xtrain, svc_ytrain = svc_loadImages("dataset/dataset/Training")
svc_Xval, svc_yval = svc_loadImages("dataset/dataset/Validation")
svc_Xtest, svc_ytest = svc_loadImages("dataset/dataset/Testing")

print(f"Datos de entrenamiento: {len(scv_Xtrain)}\tPorcentaje: {(len(scv_Xtrain)*100/1120):.2f}")
print(f"Datos de validación: {len(svc_Xval)}\tPorcentaje: {(len(svc_Xval)*100/1120):.2f}")
print(f"Datos de prueba: {len(svc_Xtest)}\t\tPorcentaje: {(len(svc_Xtest)*100/1120):.2f}")


#Escalado de imagenes con StandarScaler
scaler = StandardScaler()

scv_Xtrain_st = scaler.fit_transform(scv_Xtrain)
scv_Xval_st = scaler.transform(svc_Xval)
scv_Xtest_st = scaler.transform(svc_Xtest)

scv_Xtrain[:2], scv_Xtrain_st[:2]


#Reducir dimensionalidad con PCA a 90 componentes (componentes originales = 250)
pca = PCA(n_components=90)

scv_Xtrain_pca = pca.fit_transform(scv_Xtrain_st)
scv_Xval_pca = pca.transform(scv_Xval_st)
scv_Xtest_pca = pca.transform(scv_Xtest_st)

#### Entrenamiento del modelo svc

#### Exportación de recursos (modelo, scaler, etc)

In [ ]:
# Código

### MobileNetV2 (CNN) (keras)

In [ ]:
#algunas importaciones
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Pre-procesamiento de la data

In [ ]:
#en construcion#####
datadir = "/content/dataset/dataset/Training"
imgsize = (250,250)
batchsize = (32)

datagen = ImageDataGenerator(rescale= 1/255, validation_split=0.2)

train_data = datagen.flow_from_directory(data_dir,target_size=img_size,
                                         batch_size=batch_size,class_mode='binary',subset='training')
val_data = datagen.flow_from_directory(data_dir,target_size=img_size,
                                       batch_size=batch_size,class_mode='binary',subset='validation')


Instanciación del modelo (preentrenado)

In [ ]:
mnet = keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False